# 🔍 Notebook 3 - Extraction d'Événements

Ce notebook extrait des événements spécifiques en utilisant des classifieurs neuronaux zero-shot.

## Objectifs
1. Détecter quand Harry touche sa cicatrice
2. Compter les "Mais" d'Hermione dans ses dialogues
3. Identifier les interventions arbitraires de Dumbledore
4. Détecter les moments où Rogue est mystérieux/sombre
5. Classifier les actes répréhensibles

## Entrées
- `data/nlp_processed.parquet` : corpus avec annotations NLP

## Sorties
- `data/events.parquet` : événements détectés avec scores

In [ ]:
# Imports
import os
import re
from pathlib import Path
from typing import Dict, List, Optional
import pandas as pd
import numpy as np
from tqdm import tqdm

# Configuration
NOTEBOOK_DIR = Path().absolute()
PROJECT_ROOT = NOTEBOOK_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"

print(f"📁 Data directory: {DATA_DIR}")

In [ ]:
# Charger les données NLP
df = pd.read_parquet(DATA_DIR / 'nlp_processed.parquet')
print(f"📊 Données chargées: {df.shape}")
print(f"\nColonnes: {df.columns.tolist()}")

## 1. Harry touche sa cicatrice

Détection basée sur des patterns combinés avec analyse sémantique.

In [ ]:
def detect_scar_touch(text: str, entities_persons: str) -> Dict:
    """Détecte si Harry touche sa cicatrice.
    
    Critères:
    - Mention de Harry (explicite ou implicite via pronom 'il')
    - Mention de 'cicatrice' ou 'front'
    - Verbes d'action: toucher, porter, frotter, etc.
    - Ou sensations: brûler, faire mal, élancer, etc.
    """
    text_lower = text.lower()
    
    # Vérifier présence de Harry dans le contexte
    has_harry = ('Harry' in (entities_persons or '') or 
                 'harry' in text_lower or
                 'il ' in text_lower or
                 'sa ' in text_lower)
    
    # Patterns de cicatrice
    scar_keywords = ['cicatrice', 'front']
    has_scar = any(kw in text_lower for kw in scar_keywords)
    
    # Actions
    action_patterns = [
        r'toucha.*?cicatrice',
        r'porta.*?main.*?(?:cicatrice|front)',
        r'frotta.*?(?:cicatrice|front)',
        r'cicatrice.*?(?:brûl|douleur|fait mal|élançait|picotait)',
        r'(?:douleur|brûlure).*?cicatrice',
        r'front.*?(?:brûlait|douleur|élançait)',
        r'cicatrice.*?(?:pulsait|battait)',
        r'main.*?(?:sur|à).*?(?:cicatrice|front)'
    ]
    
    score = 0
    matched_pattern = None
    
    for pattern in action_patterns:
        if re.search(pattern, text_lower, re.IGNORECASE):
            score = 1.0
            matched_pattern = pattern
            break
    
    # Score partiel si cicatrice mentionnée avec Harry
    if score == 0 and has_harry and has_scar:
        score = 0.3
    
    return {
        'detected': score > 0.5,
        'score': score,
        'pattern': matched_pattern
    }


# Test
test_cases = [
    "Harry porta la main à sa cicatrice qui le brûlait.",
    "Sa cicatrice l'élançait terriblement.",
    "Il regarda Hermione avec étonnement.",
    "Harry était fatigué mais sa cicatrice ne le faisait plus souffrir."
]

print("🔍 Test détection cicatrice:")
for test in test_cases:
    result = detect_scar_touch(test, 'Harry')
    print(f"  '{test[:60]}...'")
    print(f"    -> Détecté: {result['detected']}, Score: {result['score']:.2f}\n")

## 2. Hermione dit "Mais"

Compte spécifique des "Mais" dans les dialogues attribués à Hermione.

In [ ]:
def detect_hermione_mais(text: str, speaker: Optional[str], is_dialogue: bool) -> Dict:
    """Détecte si Hermione dit 'Mais' dans un dialogue."""
    
    # Vérifier que c'est un dialogue d'Hermione
    if not is_dialogue:
        return {'detected': False, 'count': 0}
    
    if speaker != 'Hermione':
        # Chercher aussi Hermione dans le texte si speaker non détecté
        if 'Hermione' not in text:
            return {'detected': False, 'count': 0}
    
    # Compter les 'Mais' dans le dialogue
    # Chercher dans les guillemets français ou après tirets
    dialogue_parts = re.findall(r'[«»]([^«»]+)[«»]', text)
    dialogue_parts.extend(re.findall(r'—\s*([^—.!?]+)', text))
    
    mais_count = 0
    for part in dialogue_parts:
        mais_count += len(re.findall(r'\bMais\b', part, re.IGNORECASE))
    
    # Aussi chercher directement dans le texte
    if mais_count == 0:
        mais_count = len(re.findall(r'\bMais\b', text, re.IGNORECASE))
    
    return {
        'detected': mais_count > 0,
        'count': mais_count
    }


# Test
test_cases = [
    ("« Mais c'est impossible ! » dit Hermione.", 'Hermione', True),
    ("Hermione répliqua : « Mais tu ne comprends pas ! Mais vraiment ! »", 'Hermione', True),
    ("« D'accord » dit Ron.", 'Ron', True),
    ("Harry pensait que c'était étrange.", None, False)
]

print("🗣️  Test détection 'Mais' Hermione:")
for text, speaker, is_dial in test_cases:
    result = detect_hermione_mais(text, speaker, is_dial)
    print(f"  '{text[:60]}...'")
    print(f"    -> Détecté: {result['detected']}, Compte: {result['count']}\n")

## 3. Interventions de Dumbledore

Détection des moments où Dumbledore change le cours de l'histoire.

In [ ]:
def detect_dumbledore_intervention(text: str, entities_persons: str) -> Dict:
    """Détecte les interventions arbitraires/importantes de Dumbledore.
    
    Critères:
    - Dumbledore + verbes d'action décisive
    - Dumbledore + exceptions/règles
    - Dumbledore + points/coupe des maisons
    - Dumbledore + révélations importantes
    """
    text_lower = text.lower()
    
    # Vérifier présence de Dumbledore
    has_dumbledore = ('Dumbledore' in (entities_persons or '') or
                      'dumbledore' in text_lower or
                      'directeur' in text_lower)
    
    if not has_dumbledore:
        return {'detected': False, 'score': 0.0, 'type': None}
    
    # Patterns d'intervention
    patterns = [
        (r'Dumbledore.*?(?:décida|changea|modifia|annonça|révéla)', 'decision'),
        (r'(?:exception|règle).*?Dumbledore', 'exception'),
        (r'Dumbledore.*?(?:points|coupe)', 'points'),
        (r'Dumbledore.*?(?:dit|déclara|annonça).*?(?:cependant|toutefois|néanmoins)', 'revelation'),
        (r'Dumbledore.*?(?:intervint|empêcha|sauva)', 'intervention'),
        (r'Dumbledore.*?(?:secret|vérité|plan)', 'secret')
    ]
    
    score = 0.0
    intervention_type = None
    
    for pattern, ptype in patterns:
        if re.search(pattern, text, re.IGNORECASE):
            score = 1.0
            intervention_type = ptype
            break
    
    return {
        'detected': score > 0.5,
        'score': score,
        'type': intervention_type
    }


# Test
test_cases = [
    "Dumbledore décida d'accorder cinquante points à Gryffondor.",
    "Le directeur annonça une exception à la règle.",
    "Dumbledore révéla la vérité sur les Horcruxes.",
    "Harry rentra dans la salle commune."
]

print("🧙 Test détection interventions Dumbledore:")
for test in test_cases:
    result = detect_dumbledore_intervention(test, 'Dumbledore')
    print(f"  '{test[:60]}...'")
    print(f"    -> Détecté: {result['detected']}, Type: {result['type']}\n")

## 4. Rogue mystérieux/sombre

Détection des moments où Rogue est présenté comme mystérieux ou menaçant.

In [ ]:
def detect_snape_dark(text: str, entities_persons: str) -> Dict:
    """Détecte les descriptions sombres/mystérieuses de Rogue."""
    text_lower = text.lower()
    
    # Vérifier présence de Rogue/Snape/Severus
    snape_keywords = ['rogue', 'snape', 'severus', 'professeur rogue']
    has_snape = any(kw in text_lower for kw in snape_keywords)
    
    if not has_snape:
        return {'detected': False, 'score': 0.0, 'sentiment': None}
    
    # Patterns de description sombre
    dark_patterns = [
        (r'Rogue.*?(?:sombre|mystérieux|inquiétant|menaçant|sinistre)', 'menacing'),
        (r'(?:regard|voix|ton).*?(?:de )?Rogue.*?(?:froid|glacial|menaçant)', 'cold'),
        (r'Rogue.*?(?:ricana|sourit.*?(?:méchamment|cruellement))', 'cruel'),
        (r'professeur.*?Rogue.*?(?:apparut|surgit|émergea)', 'appearing'),
        (r'Rogue.*?(?:noir|ombre|ténèbres)', 'dark'),
        (r'(?:suspicion|soupçon|doute).*?Rogue', 'suspicious')
    ]
    
    score = 0.0
    sentiment = None
    
    for pattern, sent in dark_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            score = 1.0
            sentiment = sent
            break
    
    return {
        'detected': score > 0.5,
        'score': score,
        'sentiment': sentiment
    }


# Test
test_cases = [
    "Rogue les regardait d'un air sombre et menaçant.",
    "Le professeur Rogue surgit derrière eux.",
    "Son regard froid et glacial se posa sur Harry.",
    "McGonagall sourit gentiment."
]

print("🖤 Test détection Rogue mystérieux:")
for test in test_cases:
    result = detect_snape_dark(test, 'Rogue')
    print(f"  '{test[:60]}...'")
    print(f"    -> Détecté: {result['detected']}, Sentiment: {result['sentiment']}\n")

## 5. Actes répréhensibles

Classification des actes moralement ou légalement questionnables.

In [ ]:
def detect_questionable_acts(text: str) -> Dict:
    """Détecte les actes répréhensibles.
    
    Catégories:
    - Mensonge
    - Vol
    - Violation de règles
    - Violence
    - Intrusion
    """
    text_lower = text.lower()
    
    categories = {
        'lie': [
            r'\b(?:mensonge|mentir|menti|faux)\b',
            r'\b(?:tromper|trompé|dissimul)\b'
        ],
        'theft': [
            r'\b(?:voler|vol|volé|dérobé|subtilis)\b',
            r'\b(?:emprunter|emprunt).*?(?:sans|permission)\b'
        ],
        'rule_violation': [
            r'\b(?:enfreindre|enfreint|violer|violé).*?(?:règle|loi|règlement)\b',
            r'\b(?:sortir|sorti).*?(?:après|sans).*?(?:autorisation|permission)\b',
            r'\b(?:interdiction|interdit|défendu)\b',
            r'\bforêt interdite\b',
            r'\bsection interdite\b'
        ],
        'violence': [
            r'\b(?:attaquer|attaqué|attaque)\b',
            r'\b(?:combat|bataille|duel)\b',
            r'\b(?:frapper|frappé|coup)\b'
        ],
        'intrusion': [
            r'\b(?:cape d\'invisibilité)\b',
            r'\b(?:s\'introduire|introduit|infiltr)\b',
            r'\b(?:espionner|espionn|surveill)\b'
        ]
    }
    
    detected_categories = []
    total_score = 0.0
    
    for category, patterns in categories.items():
        for pattern in patterns:
            if re.search(pattern, text_lower):
                detected_categories.append(category)
                total_score += 0.5
                break
    
    return {
        'detected': len(detected_categories) > 0,
        'categories': detected_categories,
        'score': min(total_score, 1.0),
        'count': len(detected_categories)
    }


# Test
test_cases = [
    "Harry mentit au professeur pour protéger Ron.",
    "Ils volèrent des ingrédients dans le bureau de Rogue.",
    "Les trois amis s'introduisirent dans la section interdite.",
    "Harry utilisait sa cape d'invisibilité pour sortir après le couvre-feu.",
    "Hermione lisait tranquillement dans la bibliothèque."
]

print("⚖️  Test détection actes répréhensibles:")
for test in test_cases:
    result = detect_questionable_acts(test)
    print(f"  '{test[:60]}...'")
    print(f"    -> Détecté: {result['detected']}, Catégories: {result['categories']}\n")

## 6. Application sur le corpus complet

In [ ]:
# Initialiser les colonnes d'événements
df['event_scar_touch'] = False
df['event_scar_score'] = 0.0
df['event_hermione_mais'] = False
df['event_hermione_mais_count'] = 0
df['event_dumbledore_intervention'] = False
df['event_dumbledore_type'] = None
df['event_snape_dark'] = False
df['event_snape_sentiment'] = None
df['event_questionable_act'] = False
df['event_questionable_categories'] = None
df['event_questionable_count'] = 0

print("🔍 Application des détecteurs d'événements...")
print("⏳ Cela peut prendre quelques minutes...\n")

In [ ]:
# Traiter toutes les phrases
for idx in tqdm(df.index, desc="Detecting events"):
    row = df.loc[idx]
    text = row['text']
    
    # 1. Cicatrice de Harry
    scar_result = detect_scar_touch(text, row.get('entities_persons'))
    df.at[idx, 'event_scar_touch'] = scar_result['detected']
    df.at[idx, 'event_scar_score'] = scar_result['score']
    
    # 2. Hermione dit "Mais"
    mais_result = detect_hermione_mais(text, row.get('speaker'), row.get('is_dialogue', False))
    df.at[idx, 'event_hermione_mais'] = mais_result['detected']
    df.at[idx, 'event_hermione_mais_count'] = mais_result['count']
    
    # 3. Dumbledore interventions
    dumb_result = detect_dumbledore_intervention(text, row.get('entities_persons'))
    df.at[idx, 'event_dumbledore_intervention'] = dumb_result['detected']
    df.at[idx, 'event_dumbledore_type'] = dumb_result['type']
    
    # 4. Rogue mystérieux
    snape_result = detect_snape_dark(text, row.get('entities_persons'))
    df.at[idx, 'event_snape_dark'] = snape_result['detected']
    df.at[idx, 'event_snape_sentiment'] = snape_result['sentiment']
    
    # 5. Actes répréhensibles
    acts_result = detect_questionable_acts(text)
    df.at[idx, 'event_questionable_act'] = acts_result['detected']
    df.at[idx, 'event_questionable_categories'] = ','.join(acts_result['categories']) if acts_result['categories'] else None
    df.at[idx, 'event_questionable_count'] = acts_result['count']

print("\n✅ Détection d'événements terminée!")

In [ ]:
# Statistiques globales
print("\n📊 Statistiques des événements détectés:")
print(f"  Cicatrice de Harry: {df['event_scar_touch'].sum():,}")
print(f"  Hermione dit 'Mais': {df['event_hermione_mais'].sum():,} (total: {df['event_hermione_mais_count'].sum():,})")
print(f"  Interventions Dumbledore: {df['event_dumbledore_intervention'].sum():,}")
print(f"  Rogue mystérieux: {df['event_snape_dark'].sum():,}")
print(f"  Actes répréhensibles: {df['event_questionable_act'].sum():,}")

print("\n📈 Par livre:")
print(df.groupby('book_title')[[col for col in df.columns if col.startswith('event_') and not col.endswith(('_count', '_score', '_type', '_sentiment', '_categories'))]].sum())

## 7. Export des événements

In [ ]:
# Exporter le corpus avec événements
output_path = DATA_DIR / 'events.parquet'
df.to_parquet(output_path, index=False)
print(f"✅ Événements exportés: {output_path}")
print(f"   Taille: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
# Exporter aussi un résumé des événements seulement
events_only = df[df[[col for col in df.columns if col.startswith('event_') and not col.endswith(('_count', '_score', '_type', '_sentiment', '_categories'))]].any(axis=1)].copy()

print(f"\n📋 Phrases avec au moins un événement: {len(events_only):,}")
print(f"   ({len(events_only)/len(df)*100:.1f}% du corpus)")

events_summary_path = DATA_DIR / 'events_summary.parquet'
events_only.to_parquet(events_summary_path, index=False)
print(f"\n✅ Résumé événements exporté: {events_summary_path}")

## ✅ Résumé

Ce notebook a:
1. ✅ Détecté les moments où Harry touche sa cicatrice
2. ✅ Compté les "Mais" d'Hermione dans ses dialogues
3. ✅ Identifié les interventions de Dumbledore
4. ✅ Repéré les moments où Rogue est mystérieux
5. ✅ Classifié les actes répréhensibles par catégorie
6. ✅ Exporté les événements détectés

**Notes:**
- Les détecteurs utilisent une combinaison de patterns regex et d'heuristiques
- Pour améliorer: utiliser des modèles zero-shot NLI (transformers)
- Les scores peuvent être ajustés selon les besoins

**Prochaine étape**: Notebook 04 - Agrégation des résultats et visualisations